In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2


data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.4),  
    tf.keras.layers.RandomZoom(0.4),
    tf.keras.layers.RandomContrast(0.3),
])

def load_dataset(tfrecord_file, batch_size=32, shuffle=True, repeat=True):
    raw_dataset = tf.data.TFRecordDataset([tfrecord_file])
    feature_description = {'image': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.int64)}

    def parse_examples(serialized_examples):
        examples = tf.io.parse_example(serialized_examples, feature_description)
        targets = tf.cast(examples.pop('label'), tf.int64) 

        images = tf.image.decode_jpeg(examples['image'], channels=3)
        images = tf.image.resize_with_pad(tf.cast(images, tf.float32), 299, 299) / 255.0 
        images = data_augmentation(images)
        return images, targets

    dataset = raw_dataset.map(parse_examples, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(1000)
    if repeat:
        dataset = dataset.repeat()

    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


train_dataset = load_dataset('birds-20-eachOf-256.tfrecords')
val_dataset = load_dataset('birds-10-eachOf-256.tfrecords', shuffle=False, repeat=False)

steps_per_epoch = 5120 // 32
validation_steps = 2560 // 32

# ResNet50V2 
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False

# Custom Layers
x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dropout(0.6)(x) 
x = Dense(1024, activation='relu')(x)  
x = BatchNormalization()(x)
x = Dropout(0.6)(x)
out = Dense(256, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=out)

def clr_schedule(epoch):
    max_lr = 0.01 
    min_lr = 0.0001
    cycle = epoch % 10
    lr = min_lr + (max_lr - min_lr) * abs((cycle / 5) - 1)
    return lr

lr_callback = LearningRateScheduler(clr_schedule)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
lr_decay = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, validation_data=val_dataset, epochs=15, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
          callbacks=[lr_callback, lr_decay, early_stopping])

2025-02-27 03:16:03.574567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740644163.592674    5356 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740644163.597501    5356 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 03:16:03.620249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1740644169.061754    5356 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 55

Epoch 1/15


2025-02-27 03:16:14.013568: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1740644175.756179    5448 service.cc:148] XLA service 0x7f93c4002d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740644175.756234    5448 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-02-27 03:16:15.939899: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740644176.663028    5448 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740644185.590960    5448 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


160/160 ━━━━━━━━━━━━━━━━━━━━ 42s 169ms/step - accuracy: 0.0213 - loss: 6.4950 - val_accuracy: 0.1156 - val_loss: 4.4649 - learning_rate: 0.0100
Epoch 2/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 158ms/step - accuracy: 0.1216 - loss: 4.6905 - val_accuracy: 0.1516 - val_loss: 4.1399 - learning_rate: 0.0080
Epoch 3/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 41s 257ms/step - accuracy: 0.1949 - loss: 3.8953 - val_accuracy: 0.1793 - val_loss: 3.9245 - learning_rate: 0.0060
Epoch 4/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 155ms/step - accuracy: 0.2358 - loss: 3.5388 - val_accuracy: 0.2043 - val_loss: 3.7722 - learning_rate: 0.0041
Epoch 5/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.2876 - loss: 3.1999 - val_accuracy: 0.2078 - val_loss: 3.7207 - learning_rate: 0.0021
Epoch 6/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 23s 145ms/step - accuracy: 0.2963 - loss: 3.1695 - val_accuracy: 0.2133 - val_loss: 3.7546 - learning_rate: 1.0000e-04
Epoch 7/15
160/160 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.3073 - l

In [2]:
# Fine Tuning
base_model.trainable = True
for layer in base_model.layers[:150]: 
    layer.trainable = False

model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_dataset, validation_data=val_dataset, epochs=25, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
          callbacks=[lr_callback, lr_decay, early_stopping])

Epoch 1/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 42s 177ms/step - accuracy: 0.2032 - loss: 4.0094 - val_accuracy: 0.0762 - val_loss: 20.9085 - learning_rate: 0.0100
Epoch 2/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 41s 260ms/step - accuracy: 0.2300 - loss: 3.6021 - val_accuracy: 0.1895 - val_loss: 4.1114 - learning_rate: 0.0080
Epoch 3/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 164ms/step - accuracy: 0.3560 - loss: 2.7022 - val_accuracy: 0.2594 - val_loss: 3.4763 - learning_rate: 0.0060
Epoch 4/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 52s 324ms/step - accuracy: 0.4496 - loss: 2.2001 - val_accuracy: 0.3086 - val_loss: 3.1294 - learning_rate: 0.0041
Epoch 5/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 36s 223ms/step - accuracy: 0.5298 - loss: 1.7883 - val_accuracy: 0.3187 - val_loss: 3.0514 - learning_rate: 0.0021
Epoch 6/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 28s 175ms/step - accuracy: 0.5737 - loss: 1.6287 - val_accuracy: 0.3473 - val_loss: 2.9788 - learning_rate: 1.0000e-04
Epoch 7/25
160/160 ━━━━━━━━━━━━━━━━━━━━ 28s 173ms/step - accuracy

In [3]:
model.save('birderModel.keras')

In [4]:
import tensorflow as tf
raw_dataset = tf.data.TFRecordDataset(['birds-10-eachOf-256.tfrecords'])

feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64) 
}
def parse_examples(serialized_examples):
    examples = tf.io.parse_example(serialized_examples, feature_description)
    targets = tf.cast(examples.pop('label'), tf.int64) 
    images = tf.image.decode_jpeg(examples['image'], channels=3)
    images = tf.image.resize_with_pad(tf.cast(images, tf.float32), 299, 299) / 255.0  
    return images, targets

dataset = raw_dataset.map(parse_examples, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(128)

model = tf.keras.models.load_model('birderModel.keras')

# Performance Metrics
top5err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5')
top10err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name='top10')
top20err = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20, name='top20')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
              metrics=['accuracy', top5err, top10err, top20err])

resp = model.evaluate(dataset)

# Results
print("\n Evaluation Metrics:")
print(f"🔹 Loss: {resp[0]:.4f}")
print(f"🔹 Accuracy: {resp[1] * 100:.2f}%")
print(f"🔹 Top-5 Accuracy: {resp[2] * 100:.2f}%")
print(f"🔹 Top-10 Accuracy: {resp[3] * 100:.2f}%")
print(f"🔹 Top-20 Accuracy: {resp[4] * 100:.2f}%")


2025-02-27 03:30:10.623826: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[128,64,75,75]{3,2,1,0}, u8[0]{0}) custom-call(f32[128,64,75,75]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-02-27 03:30:10.814213: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.019265757s
Trying algorithm eng0{} for conv (f32[128,64,75,75]{3,2,1,0}, u8[0]{0}) custom-call(f32[128,64,75,75]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNo

20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 265ms/step - accuracy: 0.3946 - loss: 2.6435 - top10: 0.7882 - top20: 0.8817 - top5: 0.6752

 Evaluation Metrics:
🔹 Loss: 2.7899
🔹 Accuracy: 37.07%
🔹 Top-5 Accuracy: 65.55%
🔹 Top-10 Accuracy: 76.99%
🔹 Top-20 Accuracy: 86.68%


2025-02-27 03:30:30.494820: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-02-27 03:30:30.494934: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
/home/akhil_tom/miniconda3/envs/AppliedML_tfcuda/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
